In [ ]:
import onnx
import onnx_graphsurgeon as gs
import onnxruntime as ort
import torch
import numpy as np
from rich import print
from onnx import TensorProto

In [88]:
data_np = np.array([[1.0, 2.0, 3.0, 4.0, 5.0]],dtype=np.float32)
indices_np = np.array([[1, 3]],dtype=np.int64)
updates_np = np.array([[1.1, 2.1]],dtype=np.float32)

In [2]:
data_np = np.array([[0.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0]], dtype=np.float32)

indices_np = np.array([[1, 0, 2],
                            [0, 2, 1]], dtype=np.int64)

updates_np = np.array([[1.0, 1.1, 1.2],
                       [2.0, 2.1, 2.2]], dtype=np.float32)


In [6]:
data_np.ndim,indices_np.ndim,updates_np.ndim

(2, 2, 2)

In [90]:
data=gs.Variable(name="data",dtype=TensorProto.FLOAT,shape=[3,3])
indices=gs.Variable(name="indices",dtype=TensorProto.INT64,shape=[2,3])
updates=gs.Variable(name="updates",dtype=TensorProto.FLOAT,shape=[2,3])

output=gs.Variable(name="output",dtype=TensorProto.FLOAT,shape=[3,3])

scatter_node =gs.Node(op="Scatter",name="scatter_node",inputs=[data,indices,updates],outputs=[output])



graph =gs.Graph(nodes=[scatter_node],inputs=[data,indices,updates],outputs=[output],opset=9)
model =gs.export_onnx(graph)
onnx.checker.check_model(model)
onnx.save(model,"scatter.onnx")

Scatter ND

In [91]:
graph_nodes = []
graph_inputs = []
graph_outputs = []

# Input variables
data = gs.Variable("data", dtype=TensorProto.FLOAT, shape=[3, 3])
indices = gs.Variable("indices", dtype=TensorProto.INT64, shape=[2, 3])
updates = gs.Variable("updates", dtype=TensorProto.FLOAT, shape=[2, 3])
graph_inputs.extend([data, indices, updates])

# --- 1. Create a Range node to generate column indices for each row.
# We want to create a tensor [0, 1, 2] (since data has 3 columns).
start = gs.Constant("start", np.array(0, dtype=np.int64))
limit = gs.Constant("limit", np.array(3, dtype=np.int64))
delta = gs.Constant("delta", np.array(1, dtype=np.int64))
range_node = gs.Node("Range", "range_node", inputs=[start, limit, delta],
                     outputs=[gs.Variable("col_range", dtype=TensorProto.INT64, shape=[3])])
graph_nodes.append(range_node)

# --- 2. Unsqueeze the col_range to shape [1,3] using attribute "axes"
col_range_unsqueezed = gs.Variable("col_range_unsqueezed", dtype=TensorProto.INT64, shape=[1, 3])
unsqueeze_node = gs.Node("Unsqueeze", "unsqueeze_node", inputs=[range_node.outputs[0]],
                         outputs=[col_range_unsqueezed], attrs={"axes": [0]})
graph_nodes.append(unsqueeze_node)

# --- 3. Tile the col_range_unsqueezed along axis 0 to shape [2,3].
tiles = gs.Constant("tiles", np.array([2, 1], dtype=np.int64))
col_indices = gs.Variable("col_indices", dtype=TensorProto.INT64, shape=[2, 3])
tile_node = gs.Node("Tile", "tile_node", inputs=[col_range_unsqueezed, tiles], outputs=[col_indices])
graph_nodes.append(tile_node)

# --- 4. Unsqueeze orig_indices and col_indices to shape [2,3,1] using attribute "axes"
orig_indices_unsq = gs.Variable("orig_indices_unsq", dtype=TensorProto.INT64, shape=[2, 3, 1])
unsqueeze_orig_node = gs.Node("Unsqueeze", "unsqueeze_orig_node", inputs=[indices],
                              outputs=[orig_indices_unsq], attrs={"axes": [2]})
graph_nodes.append(unsqueeze_orig_node)

col_indices_unsq = gs.Variable("col_indices_unsq", dtype=TensorProto.INT64, shape=[2, 3, 1])
unsqueeze_col_node = gs.Node("Unsqueeze", "unsqueeze_col_node", inputs=[col_indices],
                             outputs=[col_indices_unsq], attrs={"axes": [2]})
graph_nodes.append(unsqueeze_col_node)

# --- 5. Concat the unsqueezed indices along axis 2 to get shape [2,3,2].
concat_indices = gs.Variable("concat_indices", dtype=TensorProto.INT64, shape=[2, 3, 2])
concat_node = gs.Node("Concat", "concat_node", inputs=[orig_indices_unsq, col_indices_unsq],
                      outputs=[concat_indices], attrs={"axis": 2})
graph_nodes.append(concat_node)

# --- 6. Reshape the concatenated indices to shape [6,2].
reshape_shape = gs.Constant("reshape_shape", np.array([6, 2], dtype=np.int64))
scatternd_indices = gs.Variable("scatternd_indices", dtype=TensorProto.INT64, shape=[6, 2])
reshape_indices_node = gs.Node("Reshape", "reshape_indices_node", inputs=[concat_indices, reshape_shape],
                               outputs=[scatternd_indices])
graph_nodes.append(reshape_indices_node)

# --- 7. Flatten the updates from [2,3] to [6].
updates_reshape_shape = gs.Constant("updates_reshape_shape", np.array([6], dtype=np.int64))
scatternd_updates = gs.Variable("scatternd_updates", dtype=TensorProto.FLOAT, shape=[6])
reshape_updates_node = gs.Node("Reshape", "reshape_updates_node", inputs=[updates, updates_reshape_shape],
                               outputs=[scatternd_updates])
graph_nodes.append(reshape_updates_node)

# --- 8. Create the ScatterND node.
output = gs.Variable("output", dtype=TensorProto.FLOAT, shape=[3, 3])
scatternd_node = gs.Node("ScatterND", "scatternd_node",
                         inputs=[data, scatternd_indices, scatternd_updates],
                         outputs=[output])
graph_nodes.append(scatternd_node)

# Set graph inputs and outputs.
graph = gs.Graph(nodes=graph_nodes, inputs=[data, indices, updates],
                 outputs=[output], opset=11)

# Export and save the model.
model = gs.export_onnx(graph)
onnx.checker.check_model(model)
onnx.save(model, "scatternd.onnx")

In [93]:

# Run the model using ONNX Runtime.
session = ort.InferenceSession("scatter.onnx")
result = session.run(None, input_feed={
    "data": data_np,
    "indices": indices_np,
    "updates": updates_np,
})
print("Output from original Scatter model:")
print(result[0])

session = ort.InferenceSession("scatternd.onnx")
result = session.run(None, input_feed={
    "data": data_np,
    "indices": indices_np,
    "updates": updates_np,
})
print("Output from modified ScatterND model:")
print(result[0])

Output from original Scatter model:

[[2.  1.1 0. ]
 [1.  0.  2.2]
 [0.  2.1 1.2]]

Output from modified ScatterND model:

[[2.  1.1 0. ]
 [1.  0.  2.2]
 [0.  2.1 1.2]]